In [1]:
# AXILiteS
# 0x000 : Control signals
#         bit 0  - ap_start (Read/Write/COH)
#         bit 1  - ap_done (Read/COR)
#         bit 2  - ap_idle (Read)
#         bit 3  - ap_ready (Read)
#         bit 7  - auto_restart (Read/Write)
#         others - reserved
# 0x004 : Global Interrupt Enable Register
#         bit 0  - Global Interrupt Enable (Read/Write)
#         others - reserved
# 0x008 : IP Interrupt Enable Register (Read/Write)
#         bit 0  - Channel 0 (ap_done)
#         bit 1  - Channel 1 (ap_ready)
#         others - reserved
# 0x00c : IP Interrupt Status Register (Read/TOW)
#         bit 0  - Channel 0 (ap_done)
#         bit 1  - Channel 1 (ap_ready)
#         others - reserved
# 0xc00 : Data signal of num_symbols_V
#         bit 31~0 - num_symbols_V[31:0] (Read/Write)
# 0xc04 : reserved
# 0x400 ~
# 0x7ff : Memory 'sorted_value_V' (256 * 32b)
#         Word n : bit [31:0] - sorted_value_V[n]
# 0x800 ~
# 0xbff : Memory 'sorted_frequency_V' (256 * 32b)
#         Word n : bit [31:0] - sorted_frequency_V[n]
# 0xd00 ~
# 0xdff : Memory 'codeword_length_histogram_V' (64 * 32b)
#         Word n : bit [31:0] - codeword_length_histogram_V[n]
# 0xe00 ~
# 0xeff : Memory 'symbol_bits_V' (256 * 5b)
#         Word n : bit [ 4: 0] - symbol_bits_V[4n]
#                  bit [12: 8] - symbol_bits_V[4n+1]
#                  bit [20:16] - symbol_bits_V[4n+2]
#                  bit [28:24] - symbol_bits_V[4n+3]
#                  others      - reserved
# (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

from pynq import Overlay

overlay = Overlay("../bitstream/canonize-tree-test.bit")
overlay.download()

canonize_tree = overlay.canonize_tree_0

# sorted table
sorted_table = [
    { 'symbol': 'f', 'freq': 1 },
    { 'symbol': 'b', 'freq': 1 },
    { 'symbol': 'c', 'freq': 2 },
    { 'symbol': 'a', 'freq': 3 },
    { 'symbol': 'd', 'freq': 5 },
    { 'symbol': 'e', 'freq': 5 }
]

bit_len = [ 0, 0, 3, 1, 2 ]

# write symbol number
canonize_tree.write(0xc00, 6)

for idx, symbol in enumerate(sorted_table):
    # write symbol
    canonize_tree.write(0x400 + 4*idx, ord(symbol['symbol']))
    # write frequency
    canonize_tree.write(0x800 + 4*idx, symbol['freq'])

for idx, bt in enumerate(bit_len):
    # write bit length
    canonize_tree.write(0xd00 + 4*idx, bt)
    
# start
canonize_tree.write(0x000, 1)

from time import sleep

sleep(1)

# 0x1f = 00011111b
word_mask = 0x1f

# search memory for coding length
for idx, symbol in enumerate(sorted_table):
    word_addr = int(ord(symbol['symbol']) / 4)
    bit_len_word = canonize_tree.read(0xe00 + 4*word_addr)
    seg_pos = ord(symbol['symbol']) % 4
    bit_len = (bit_len_word >> (seg_pos*8)) & word_mask
    print('S : ' + symbol['symbol'] + ' L : ' + str(bit_len))

S : f L : 4
S : b L : 4
S : c L : 3
S : a L : 2
S : d L : 2
S : e L : 2
